# NaonedIA - Expérience Logement

Hector Basset - Ippon Technologies

Analyse des données du cadastre en vue de la sélection d'un dataset.

In [33]:
import pandas as pd
import numpy as np
from os import path

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [34]:
py.init_notebook_mode(connected=True)

def py_table(data, filename):
    table = ff.create_table(data)
    py.iplot(table, filename=filename)

def py_pie(values, title, filename, labels=None):
    if labels is None:
        labels = values.index
        values = values.values
    pie = go.Pie(labels=labels, values=values, title=title)
    py.iplot([pie], filename=filename)

In [61]:
def get_code_postaux():
    return pd.read_csv(
        'data/code-postaux.csv',
        delimiter=';',
        header=0,
        usecols=[
            'Code_commune_INSEE'
        ],
        index_col=False,
        squeeze=True
    ).unique()

def get_data_year(year, cp):
    data_year = pd.read_csv(
        'data/dvf/44_%i.csv.gz' % year,
        header=0,
        usecols=[
            'date_mutation',
            'nature_mutation',
            'valeur_fonciere',
            'adresse_numero',
            'adresse_suffixe',
            'adresse_code_voie',
            'adresse_nom_voie',
            'code_postal',
            'nom_commune',
            'ancien_code_commune',
            'code_commune',
            'lot1_surface_carrez',
            'lot2_surface_carrez',
            'lot3_surface_carrez',
            'lot4_surface_carrez',
            'lot5_surface_carrez',
            'nombre_lots',
            'type_local',
            'surface_reelle_bati',
            'nombre_pieces_principales',
            'surface_terrain',
            'longitude',
            'latitude'
        ],
        index_col=False,
        iterator=True
    )

    data_year = pd.concat([chunk[chunk['code_commune'].isin(cp) | chunk['ancien_code_commune'].isin(cp)] for chunk in data_year], ignore_index=True)

    return data_year

def get_data():
    cp = get_code_postaux()

    data = pd.concat([get_data_year(year, cp) for year in [2014, 2015, 2016, 2017, 2018]], ignore_index=True)

    data['nature_mutation'] = data['nature_mutation'].astype('category')
    data['valeur_fonciere'] = data['valeur_fonciere'].astype('float64')
    data['adresse_numero'] = data['adresse_numero'].astype('str')
    data['adresse_numero'] = data['adresse_numero'].apply(lambda n: n.split('.')[0])
    data['adresse_suffixe'] = data['adresse_suffixe'].astype('category')
    data['adresse_code_voie'] = data['adresse_code_voie'].astype('category')
    data['adresse_nom_voie'] = data['adresse_nom_voie'].astype('str')
    data['code_postal'] = data['code_postal'].astype('str')
    data['code_postal'] = data['code_postal'].apply(lambda n: n.split('.')[0])
    data['code_postal'] = data['code_postal'].astype('category')
    data['nom_commune'] = data['nom_commune'].astype('category')
    data['lot1_surface_carrez'] = data['lot1_surface_carrez'].astype('float64')
    data['lot2_surface_carrez'] = data['lot2_surface_carrez'].astype('float64')
    data['lot3_surface_carrez'] = data['lot3_surface_carrez'].astype('float64')
    data['lot4_surface_carrez'] = data['lot4_surface_carrez'].astype('float64')
    data['lot5_surface_carrez'] = data['lot5_surface_carrez'].astype('float64')
    data['nombre_lots'] = data['nombre_lots'].astype('int64')
    data['type_local'] = data['type_local'].astype('category')
    data['surface_reelle_bati'] = data['surface_reelle_bati'].astype('float64')
    data['nombre_pieces_principales'] = data['nombre_pieces_principales'].astype('float64')
    data['surface_terrain'] = data['surface_terrain'].astype('float64')
    data['longitude'] = data['longitude'].astype('float64')
    data['latitude'] = data['latitude'].astype('float64')
    data.drop(['code_commune', 'ancien_code_commune'],axis=1, inplace=True)

    return data

## Chargement des données relatives à Nantes Métropole

In [62]:
df = None
if path.isfile('data/dvf/all.pkl.xz'):
    df = pd.read_pickle('data/dvf/all.pkl.xz')
else:
    df = get_data()
    df.to_pickle('data/dvf/all.pkl.xz')

print("%i observations chargées" % df.shape[0])
df.head(10)

151591 observations chargées


,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,...,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-08,Vente,29000.00,7,NaN,AV BASCHER,0568,44000,Nantes,NaN,...,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
1,2014-01-08,Vente,29000.00,7,NaN,AV BASCHER,0568,44000,Nantes,NaN,...,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
2,2014-01-09,Vente en l'état futur d'achèvement,285000.00,29,B,RUE DE LA PELLETERIE,6300,44000,Nantes,NaN,...,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,NaN,NaN
3,2014-01-09,Vente en l'état futur d'achèvement,285000.00,29,B,RUE DE LA PELLETERIE,6300,44000,Nantes,NaN,...,NaN,NaN,NaN,1,Appartement,65.0,3.0,NaN,NaN,NaN
4,2014-01-06,Vente en l'état futur d'achèvement,251538.46,nan,NaN,nan,NaN,nan,Nantes,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,-1.533561,47.216841
5,2014-01-06,Vente en l'état futur d'achèvement,251538.46,nan,NaN,nan,NaN,nan,Nantes,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,-1.533561,47.216841
6,2014-01-07,Vente en l'état futur d'achèvement,329096.99,5,NaN,PL ARISTIDE BRIAND,0360,44000,Nantes,NaN,...,NaN,NaN,NaN,1,Appartement,73.0,3.0,NaN,-1.563677,47.216822
7,2014-01-07,Vente en l'état futur d'achèvement,329096.99,5,NaN,PL ARISTIDE BRIAND,0360,44000,Nantes,NaN,...,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.563677,47.216822
8,2014-01-08,Vente,192.00,nan,NaN,LES CLOSEAUX,B098,44800,Saint-Herblain,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,24.0,-1.629867,47.239388
9,2014-01-02,Vente,194400.00,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,...,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.603261,47.209692


In [63]:
print("%i variables" % df.shape[1])
pd.DataFrame(df.columns, columns=['Colonnes'])

21 variables


,Colonnes
0,date_mutation
1,nature_mutation
2,valeur_fonciere
3,adresse_numero
4,adresse_suffixe
5,adresse_nom_voie
6,adresse_code_voie
7,code_postal
8,nom_commune
9,lot1_surface_carrez


## Analyse des natures de mutation

In [64]:
py_pie(df.groupby(['nature_mutation']).size(), 'Natures de mutation dans le dataset', 'natures')

Ce dataset inclue des mutations autres qu'une simple vente (qui représente cependant la grande majorité des cas). Pour simplifier et se limiter aux cas d'utilisation les plus fréquents, nous n'allons garder que les ventes, et retirer la variable `Nature mutation`.

In [67]:
df.drop(df[df['nature_mutation'] != 'Vente'].index, inplace=True)
df.drop('nature_mutation', axis=1, inplace=True)

In [68]:
print("%i observations restantes" % df.shape[0])
df.head(10)

113336 observations restantes


,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-08,29000.0,7,NaN,AV BASCHER,0568,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
1,2014-01-08,29000.0,7,NaN,AV BASCHER,0568,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
8,2014-01-08,192.0,nan,NaN,LES CLOSEAUX,B098,44800,Saint-Herblain,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,24.0,-1.629867,47.239388
9,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.603261,47.209692
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN,-1.603261,47.209692
11,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,NaN,NaN,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.644254,47.207462
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,NaN,NaN,NaN,NaN,1,Appartement,46.0,2.0,NaN,-1.644254,47.207462
13,2014-01-08,295000.0,13,NaN,RUE DE L ARCHE SECHE,0348,44000,Nantes,NaN,89.2,NaN,NaN,NaN,2,Appartement,105.0,2.0,NaN,-1.558665,47.216404
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,NaN,NaN,NaN,NaN,1,Appartement,103.0,5.0,NaN,-1.519753,47.272364
15,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,NaN,NaN,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.519753,47.272364


## Analyse des types de locaux

In [9]:
py_pie(df.groupby(['Type local']).size(), 'Types de locaux dans le dataset', 'locaux')

Là aussi, pour simplifier et adresser le plus grand nombre de cas d'utilisation, nous allons nous limiter aux appartements et aux maisons.

In [69]:
df.drop(df[~df['type_local'].isin(['Appartement', 'Maison'])].index, inplace=True)
df['type_local'].cat.remove_unused_categories(inplace=True)

In [70]:
print("%i observations restantes" % df.shape[0])
df.head(10)

57567 observations restantes


,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,NaN,NaN,NaN,NaN,1,Appartement,46.0,2.0,NaN,-1.644254,47.207462
13,2014-01-08,295000.0,13,NaN,RUE DE L ARCHE SECHE,0348,44000,Nantes,NaN,89.2,NaN,NaN,NaN,2,Appartement,105.0,2.0,NaN,-1.558665,47.216404
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,NaN,NaN,NaN,NaN,1,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,NaN,NaN,NaN,NaN,1,Appartement,25.0,1.0,NaN,-1.551322,47.216626
20,2014-01-10,40000.0,21,NaN,BD VICTOR HUGO,8420,44200,Nantes,NaN,NaN,NaN,NaN,NaN,2,Appartement,19.0,1.0,NaN,-1.552176,47.204726
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,NaN,NaN,NaN,NaN,0,Maison,118.0,5.0,562.0,-1.506824,47.231122
22,2014-01-04,67000.0,5,NaN,BD HONORE DE BALZAC,4220,44100,Nantes,22.42,NaN,NaN,NaN,NaN,1,Appartement,22.0,1.0,NaN,-1.581341,47.202202
24,2014-01-06,230000.0,6,NaN,AV DES CIGALES,1905,44300,Nantes,NaN,NaN,NaN,NaN,NaN,0,Maison,99.0,4.0,269.0,-1.502877,47.232062
26,2014-01-03,160000.0,25,NaN,RUE LOUIS BROCHU,0790,44230,Saint-Sébastien-sur-Loire,NaN,NaN,NaN,NaN,NaN,0,Maison,68.0,4.0,434.0,-1.487932,47.198380


## Analyse du nombre de lots

In [71]:
py_pie(df.groupby(['nombre_lots']).size(), 'Nombres de lots de chaque bien dans le dataset', 'lots')

De nombreux biens sont vendus en lots, et la valeur foncière représente alors le total des lots. Là aussi dans un but de simplification nous n'allons garder que les biens contenant 0 ou 1 lot, et retirer les variables `Nombre de lots` et `Surface Carrez du 2eme lot` et plus.

In [72]:
df.drop(df[df['nombre_lots'] > 1].index, inplace=True)
df.drop('nombre_lots', axis=1, inplace=True)
df.drop('lot2_surface_carrez', axis=1, inplace=True)
df.drop('lot3_surface_carrez', axis=1, inplace=True)
df.drop('lot4_surface_carrez', axis=1, inplace=True)
df.drop('lot5_surface_carrez', axis=1, inplace=True)

In [73]:
print("%i observations restantes" % df.shape[0])
df.head(10)

42196 observations restantes


,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122
22,2014-01-04,67000.0,5,NaN,BD HONORE DE BALZAC,4220,44100,Nantes,22.42,Appartement,22.0,1.0,NaN,-1.581341,47.202202
24,2014-01-06,230000.0,6,NaN,AV DES CIGALES,1905,44300,Nantes,NaN,Maison,99.0,4.0,269.0,-1.502877,47.232062
26,2014-01-03,160000.0,25,NaN,RUE LOUIS BROCHU,0790,44230,Saint-Sébastien-sur-Loire,NaN,Maison,68.0,4.0,434.0,-1.487932,47.198380
30,2014-01-10,172000.0,4,NaN,BD BOULAY PATY,1116,44100,Nantes,75.64,Appartement,74.0,3.0,NaN,-1.575381,47.230107
35,2014-01-10,250000.0,2,NaN,RUE BOSSUET,1056,44000,Nantes,NaN,Appartement,19.0,1.0,NaN,-1.554965,47.216934


## Analyse des communes

In [90]:
py_pie(df.groupby(['nom_commune']).size(), 'Nombres de bien par commune dans le dataset', 'bien')

In [93]:
df['nom_commune'].unique()

[Nantes, Saint-Herblain, Saint-Sébastien-sur-Loire, Indre, Vertou, ..., Brains, La Montagne, Le Pellerin, Saint-Jean-de-Boiseau, Couëron]
Length: 24
Categories (24, object): [Nantes, Saint-Herblain, Saint-Sébastien-sur-Loire, Indre, ..., La Montagne, Le Pellerin, Saint-Jean-de-Boiseau, Couëron]

## Suppression des doublons

In [74]:
df.drop_duplicates(inplace=True)

In [75]:
print("%i observations restantes" % df.shape[0])
df.head(10)

40283 observations restantes


,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122
22,2014-01-04,67000.0,5,NaN,BD HONORE DE BALZAC,4220,44100,Nantes,22.42,Appartement,22.0,1.0,NaN,-1.581341,47.202202
24,2014-01-06,230000.0,6,NaN,AV DES CIGALES,1905,44300,Nantes,NaN,Maison,99.0,4.0,269.0,-1.502877,47.232062
26,2014-01-03,160000.0,25,NaN,RUE LOUIS BROCHU,0790,44230,Saint-Sébastien-sur-Loire,NaN,Maison,68.0,4.0,434.0,-1.487932,47.198380
30,2014-01-10,172000.0,4,NaN,BD BOULAY PATY,1116,44100,Nantes,75.64,Appartement,74.0,3.0,NaN,-1.575381,47.230107
35,2014-01-10,250000.0,2,NaN,RUE BOSSUET,1056,44000,Nantes,NaN,Appartement,19.0,1.0,NaN,-1.554965,47.216934


## Données manquantes

Observons le taux de remplissage des colonnes :

In [76]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
date_mutation,100.00 %
valeur_fonciere,99.69 %
adresse_numero,100.00 %
adresse_suffixe,6.20 %
adresse_nom_voie,100.00 %
adresse_code_voie,100.00 %
code_postal,100.00 %
nom_commune,100.00 %
lot1_surface_carrez,29.70 %
type_local,100.00 %


La valeur foncière n'est pas toujours remplie, or il s'agit de la variable que l'on cherche à prédire. On supprime donc les observations concernées.

In [78]:
df.drop(df[df['valeur_fonciere'].isna()].index, inplace=True)

In [83]:
df.drop(df[df['longitude'].isna()].index, inplace=True)

In [84]:
df.drop(df[df['latitude'].isna()].index, inplace=True)

In [85]:
print("%i observations restantes" % df.shape[0])
df.head(10)

38872 observations restantes


,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122
22,2014-01-04,67000.0,5,NaN,BD HONORE DE BALZAC,4220,44100,Nantes,22.42,Appartement,22.0,1,NaN,-1.581341,47.202202
24,2014-01-06,230000.0,6,NaN,AV DES CIGALES,1905,44300,Nantes,NaN,Maison,99.0,4,269.0,-1.502877,47.232062
26,2014-01-03,160000.0,25,NaN,RUE LOUIS BROCHU,0790,44230,Saint-Sébastien-sur-Loire,NaN,Maison,68.0,4,434.0,-1.487932,47.198380
30,2014-01-10,172000.0,4,NaN,BD BOULAY PATY,1116,44100,Nantes,75.64,Appartement,74.0,3,NaN,-1.575381,47.230107
35,2014-01-10,250000.0,2,NaN,RUE BOSSUET,1056,44000,Nantes,NaN,Appartement,19.0,1,NaN,-1.554965,47.216934


In [86]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
date_mutation,100.00 %
valeur_fonciere,100.00 %
adresse_numero,100.00 %
adresse_suffixe,6.24 %
adresse_nom_voie,100.00 %
adresse_code_voie,100.00 %
code_postal,100.00 %
nom_commune,100.00 %
lot1_surface_carrez,30.31 %
type_local,100.00 %


La surface Carrez n'est renseignée que dans moins de 30 % des cas, et la surface du terrain dans moins de 60 %.

Proposition :
* supprimer la surface Carrez, et n'utiliser que la surface batie ;
* utiliser la surface batie à la place de celle du terrain là où cette dernière n'est pas remplie.

Toutes les autres variables sont remplies à 100 %, hormis `B/T/Q` et `Type de voie` mais ça n'est pas grave car une adresse ne contient pas forcement un numéro ou un type de voie (cas des maisons isolées par exemple).

In [87]:
# Nombre pieces principales est maintenant rempli à 100 %, on peut le convertir en int
df['nombre_pieces_principales'] = df['nombre_pieces_principales'].astype('int64')

df.head(10)

,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122
22,2014-01-04,67000.0,5,NaN,BD HONORE DE BALZAC,4220,44100,Nantes,22.42,Appartement,22.0,1,NaN,-1.581341,47.202202
24,2014-01-06,230000.0,6,NaN,AV DES CIGALES,1905,44300,Nantes,NaN,Maison,99.0,4,269.0,-1.502877,47.232062
26,2014-01-03,160000.0,25,NaN,RUE LOUIS BROCHU,0790,44230,Saint-Sébastien-sur-Loire,NaN,Maison,68.0,4,434.0,-1.487932,47.198380
30,2014-01-10,172000.0,4,NaN,BD BOULAY PATY,1116,44100,Nantes,75.64,Appartement,74.0,3,NaN,-1.575381,47.230107
35,2014-01-10,250000.0,2,NaN,RUE BOSSUET,1056,44000,Nantes,NaN,Appartement,19.0,1,NaN,-1.554965,47.216934


In [88]:
df.to_pickle('store/after_analyze.pkl.xz')

## Conclusion de l'analyse

En utilisant ce dataset, un fois nettoyé, nous disposons d'une base solide de plus de 40 000 observations contenant des données réelles (non estimées par le vendeur par exemple). De plus, le taux de remplissage des variables est élevé.

Le seul bémol est le nombre un peu trop petit de variables. Cependant, il est tout à fait possible d'utiliser les informations de ce dataset pour l'enrichir et ajouter des variables qui pourraient être pertinentes. Je fais 2 propositions dans ce sens ci-dessous.

## Exploitation des adresses

Les adresses sont renseignées avec précision. Nous pourrions donc les utiliser afin d'enrichir le dataset de variables telles que : Distance de l'arrêt de tram le plus proche, de l'arrêt de bus, de la gare, d'une pharmacie, d'un supermarché, d'un groupe de commerces, etc. Le choix des variables est à arrêter en fonction des informations que l'on souhaite demander à l'utilisateur et de ce qu'il sera possible de faire pour enrichir le dataset via OpenStreetMap par exemple. Il peut également être intéressant de renseigner les distances à pied ou en voiture.

Une fois le dataset enrichi, nous supprimerons les adresses des données car le but est de ne pas stocker des informations trop précises (pour ne pas inquiéter l'utilisateur).

Pour éviter à l'utilisateur d'avoir à saisir toutes ces informations, on pourrais lui demander de saisir directement l'adresse du bien, en lui précisant bien qu'elle ne sera ni stockée ni utilisée pour la prédiction mais uniquement pour déduire les informations citées plus haut (et avec plus de précision que si c'est lui qui le fait du coup).

Nantes Métropole fournit de nombreux datasets en open data que l'on pourrai utiliser :
* [localisation des arrêts de tram et de bus](https://data.nantesmetropole.fr/explore/dataset/244400404_tan-arrets/table/) ;
* [localisation des entreprises](https://data.nantesmetropole.fr/explore/dataset/244400404_base-sirene-entreprises-nantes-metropole/table/) ;
* ...

## Exploitation de la date

Les dates de vente sont elles aussi renseignées avec précision. Nous pourrions les utiliser afin d'enrichir le dataset de variables telles que : prix moyen du mètre carré à cette date, tension du marché à cette date, revenu moyen par habitant à cette date, densité de population, etc. À renseigner pour l'année courante ou le mois courant (ou précédent), au niveau national ou local en fonction de ce que l'on peut trouver comme historique sur le net.

L'utilisateur n'aura bien entendu pas à rentrer ces données lorsqu'il voudra estimer un bien, mais on viendrait automatiquement enrichir sa saisie avec les derniers chiffres connus.